## 文本相似度text_similarity----cross-model交互策略
本质上也是分类任务

## Step1 导入相关包

In [1]:
import os

# 设置可见的 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [2]:
dataset = load_dataset("json", data_files="train_pair_1w.json", split="train")
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

## Step3 划分数据集


In [ ]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

## Step4 数据集预处理

In [4]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_function(examples):
    tokenized_examples = tokenizer(examples["sentence1"], examples["sentence2"], max_length=128, truncation=True)
    tokenized_examples["labels"] = [float(label) for label in examples["label"]]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [5]:
print(tokenized_datasets["train"][0])

{'input_ids': [101, 4511, 2094, 6716, 4959, 3340, 5292, 6137, 6136, 8024, 1381, 5504, 2806, 3312, 5074, 102, 1343, 3378, 702, 1765, 3175, 4638, 782, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0.0}


## Step5 创建模型

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step6 创建评估函数

In [7]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [8]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = [1 if p > 0.5 else 0 for p in predictions]
    labels = [int(label) for label in labels]
    # predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainArguments

In [9]:
train_args = TrainingArguments(output_dir="./cross_model",      # 输出文件夹
                               per_device_train_batch_size=32,  # 训练时的batch_size
                               per_device_eval_batch_size=32,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=2,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True      # 训练完成后加载最优模型
                               )     
train_args

/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
f

## Step8 创建Trainer

In [10]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型训练

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.109900,0.069796,0.907000,0.883750
2,0.074100,0.061060,0.918500,0.898569
3,0.064400,0.063728,0.920000,0.900374


TrainOutput(global_step=750, training_loss=0.08847727115948995, metrics={'train_runtime': 147.831, 'train_samples_per_second': 162.348, 'train_steps_per_second': 5.073, 'total_flos': 1553919940810752.0, 'train_loss': 0.08847727115948995, 'epoch': 3.0})

## Step10 模型评估

In [12]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.06372786313295364,
 'eval_accuracy': 0.92,
 'eval_f1': 0.900373599003736,
 'eval_runtime': 3.373,
 'eval_samples_per_second': 592.945,
 'eval_steps_per_second': 18.678,
 'epoch': 3.0}

In [13]:
trainer.evaluate(tokenized_datasets["train"])

{'eval_loss': 0.020500192418694496,
 'eval_accuracy': 0.97725,
 'eval_f1': 0.9713295526149969,
 'eval_runtime': 13.566,
 'eval_samples_per_second': 589.711,
 'eval_steps_per_second': 18.428,
 'epoch': 3.0}

## Step11 模型预测

In [32]:
from transformers import pipeline

In [33]:
model.config.id2label = {0:"不相似", 1:"相似"}

In [34]:
pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipeline

In [49]:
result = pipeline({"text": "我喜欢北京。", "text_pair": "天气怎样？"}, function_to_apply="none")
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '不相似', 'score': -0.004505789838731289}

In [48]:
result = pipeline({"text": "我喜欢北京。", "text_pair": "北京是我喜欢的地方。"}, function_to_apply="none")
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '相似', 'score': 1.0234427452087402}

## 训练过程可视化
1、终端进入abc的conda环境和checkpoints目录，执行tensorboard --logdir=runs --host=0.0.0.0 --port=8418

2、vscode中ctrl+shift+p，搜索TensorBoard